In [8]:
!pip install pyDOE

#you need to add a dataset with GaussJacobiQuadRule_V3 on the right 
import sys
# sys.path.insert(1, '/kaggle/input/quadrule')
from GaussJacobiQuadRule_V3 import Jacobi, DJacobi, GaussLobattoJacobiWeights, GaussJacobiWeights

#import GaussJacobiQuadRule_V3


import tensorflow as tf
print(tf.version)
#tf.disable_v2_behavior()
import pyDOE

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from pyDOE import lhs
#from GaussJacobiQuadRule_V3.py import Jacobi, DJacobi, GaussLobattoJacobiWeights, GaussJacobiWeights
import time

<module 'tensorflow._api.v2.version' from '/home/mariano/Documenti/test/base/lib/python3.9/site-packages/tensorflow/_api/v2/version/__init__.py'>


In [9]:
N_Quad=100
N_testfcn=20
N_Element=5
a=-1.0
b=1.0

omega = 8*np.pi
amp = 1.0
r1 = 80.0

def u_exact(x):
    utemp = 0.1*np.sin(omega*x) + np.tanh(r1*x)
    return amp*utemp

def f_ext(x):
    gtemp =  -0.1*(omega**2)*np.sin(omega*x) - (2*r1**2)*(np.tanh(r1*x))/((np.cosh(r1*x))**2)
    return -amp*gtemp

def Test_f(n,x):
    test  = Jacobi(n+1,0,0,x) - Jacobi(n-1,0,0,x)
    return test

def Test_fcn(N_test,x):
    test_total = []
    for n in range(1,N_test+1):  
        test  = Jacobi(n+1,0,0,x) - Jacobi(n-1,0,0,x)
        test_total.append(test)
    return np.asarray(test_total)

def dTest_fcn(N_test,x):  #valuete the first and second derivatives of test functions on a point x 
    d1test_total = []
    d2test_total = []
    for n in range(1,N_test+1):  
        if n==1:
            d1test = ((n+2)/2)*Jacobi(n,1,1,x)
            d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x)
            d1test_total.append(d1test)
            d2test_total.append(d2test)
        elif n==2:
            d1test = ((n+2)/2)*Jacobi(n,1,1,x) - ((n)/2)*Jacobi(n-2,1,1,x)
            d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x)
            d1test_total.append(d1test)
            d2test_total.append(d2test)    
        else:
            d1test = ((n+2)/2)*Jacobi(n,1,1,x) - ((n)/2)*Jacobi(n-2,1,1,x)
            d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x) - ((n)*(n+1)/(2*2))*Jacobi(n-3,2,2,x)
            d1test_total.append(d1test)
            d2test_total.append(d2test)    
    return np.asarray(d1test_total), np.asarray(d2test_total)




[x_quad,w_quad] = GaussLobattoJacobiWeights(N_Quad, 0, 0)

        

test_quad_element = Test_fcn(N_testfcn,x_quad)
d1test_quad_element, d2test_quad_element = dTest_fcn(N_testfcn,x_quad)


[x_l, x_r] = [a, b]     #modify in the future if you want general (a,b) interval
delta_x = (x_r - x_l)/N_Element
grid = np.asarray([ x_l + i*delta_x for i in range(N_Element+1)])

        #build right side (it doesnt depend on the network)

F_ext_total = []
for e in range(N_Element):
    x_quad_element = grid[e] + (grid[e+1]-grid[e])/2*(x_quad+1)  #traslation of the element 

    jacobian = (grid[e+1]-grid[e])/2
    testfcn_element = np.asarray([Test_f(n,x_quad)  for n in range(1,N_testfcn+1)])

    #this is fh calculated on the real nodal values summed over each real nodal values(which comes form CGL nodes )

    f_quad_element = f_ext(x_quad_element)
    F_ext_element  = jacobian*np.asarray([sum(w_quad*f_quad_element*testfcn_element[i]) for i in range(N_testfcn)])    
    F_ext_element = F_ext_element[:,None] 
    F_ext_total.append(F_ext_element)

F_ext_total = np.asarray(F_ext_total)
X_bound = np.asarray([a,b],dtype=np.float64)[:,None]
u_bound   =u_exact(X_bound)


X_bound = tf.constant(X_bound,dtype=tf.float64)
u_bound   =tf.constant(u_bound,dtype=tf.float64)

F_ext_total=tf.constant(F_ext_total,dtype=tf.float64)

x_quad=tf.constant(x_quad,dtype=tf.float64)

x_quad=tf.expand_dims(x_quad,axis=1)

print(x_quad)




test_quad_element =tf.convert_to_tensor(Test_fcn(N_testfcn, x_quad),tf.float64)
d1test_quad_element, d2test_quad_element = tf.convert_to_tensor(dTest_fcn(N_testfcn, x_quad),tf.float64)

d1test_bound_element, d2test_bound_element = tf.convert_to_tensor(dTest_fcn(N_testfcn, np.array([[-1],[1]])),tf.float64)

tf.Tensor(
[[-1.        ]
 [-0.99925858]
 [-0.99751525]
 [-0.99477731]
 [-0.99104769]
 [-0.98633016]
 [-0.98062943]
 [-0.97395118]
 [-0.96630207]
 [-0.95768973]
 [-0.94812275]
 [-0.93761065]
 [-0.92616392]
 [-0.91379398]
 [-0.90051314]
 [-0.88633465]
 [-0.87127264]
 [-0.85534213]
 [-0.838559  ]
 [-0.82093997]
 [-0.80250261]
 [-0.7832653 ]
 [-0.76324721]
 [-0.7424683 ]
 [-0.72094929]
 [-0.69871161]
 [-0.67577744]
 [-0.65216965]
 [-0.62791176]
 [-0.60302795]
 [-0.57754303]
 [-0.5514824 ]
 [-0.52487204]
 [-0.49773848]
 [-0.47010876]
 [-0.44201043]
 [-0.41347149]
 [-0.3845204 ]
 [-0.35518601]
 [-0.32549756]
 [-0.29548464]
 [-0.26517718]
 [-0.23460539]
 [-0.20379974]
 [-0.17279093]
 [-0.14160988]
 [-0.11028768]
 [-0.07885553]
 [-0.04734478]
 [-0.01578684]
 [ 0.01578684]
 [ 0.04734478]
 [ 0.07885553]
 [ 0.11028768]
 [ 0.14160988]
 [ 0.17279093]
 [ 0.20379974]
 [ 0.23460539]
 [ 0.26517718]
 [ 0.29548464]
 [ 0.32549756]
 [ 0.35518601]
 [ 0.3845204 ]
 [ 0.41347149]
 [ 0.44201043]
 [ 0.47010876]

In [10]:
class VPINN(tf.Module):
    def __init__(self,F_ext_total,x_quad,w_quad,grid,test_quad_element,d1test_quad_element, d2test_quad_element,d1test_bound_element, d2test_bound_element):
        super().__init__()
        seed_value=53
        self.dense_layers1 = tf.keras.layers.Dense(20, activation=tf.sin,dtype=tf.float64,kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value))
        self.dense_layers2 = tf.keras.layers.Dense(20, activation=tf.sin,dtype=tf.float64,kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value))
        self.dense_layers3 = tf.keras.layers.Dense(20, activation=tf.sin,dtype=tf.float64,kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value))
        self.dense_layers4 = tf.keras.layers.Dense(20, activation=tf.sin,dtype=tf.float64,kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value))

        self.final_dense = tf.keras.layers.Dense(1, activation='linear',dtype=tf.float64,kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value))


        self.F_ext_total=F_ext_total
        self.x_quad=x_quad
        self.w_quad=w_quad
        self.grid=grid
        self.test_quad_element=test_quad_element
        self.d1test_quad_element,self.d2test_quad_element=d1test_quad_element, d2test_quad_element
        self.d1test_bound_element, self.d2test_bounda_element=d1test_bound_element, d2test_bound_element






    @tf.function
    def __call__(self, inputs):
        x = inputs
        x=self.dense_layers1(x)
        x=self.dense_layers2(x)
        x=self.dense_layers3(x)
        x=self.dense_layers4(x)
        output = self.final_dense(x)
        return output
    
    @tf.function
    def net_du(self,X):
        with tf.GradientTape() as tape_:
            tape_.watch(X)
            y=self(X)
        return tape_.gradient(y,X)
    


        """
        with tf.GradientTape() as tape:
            tape.watch(X)
            with tf.GradientTape() as tape_:
                tape_.watch(X)
                y=self(X)
            d1=tape_.gradient(y,X)
        d2=tape.gradient(d1,X)
        return d1,d2

        """
        
    
    def Test_fcn(self, N_test,x):
        test_total = []
        for n in range(1,N_test+1):  
            test  = Jacobi(n+1,0,0,x) - Jacobi(n-1,0,0,x)
            test_total.append(test)
        return np.asarray(test_total)

    def dTest_fcn(self, N_test,x):  #valuete the first and second derivatives of test functions on a point x 
        d1test_total = []
        d2test_total = []
        for n in range(1,N_test+1):  
            if n==1:
                d1test = ((n+2)/2)*Jacobi(n,1,1,x)
                d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x)
                d1test_total.append(d1test)
                d2test_total.append(d2test)
            elif n==2:
                d1test = ((n+2)/2)*Jacobi(n,1,1,x) - ((n)/2)*Jacobi(n-2,1,1,x)
                d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x)
                d1test_total.append(d1test)
                d2test_total.append(d2test)    
            else:
                d1test = ((n+2)/2)*Jacobi(n,1,1,x) - ((n)/2)*Jacobi(n-2,1,1,x)
                d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x) - ((n)*(n+1)/(2*2))*Jacobi(n-3,2,2,x)
                d1test_total.append(d1test)
                d2test_total.append(d2test)    
        return np.asarray(d1test_total), np.asarray(d2test_total)
    
    def Test_f(self,n,x):
        test  = Jacobi(n+1,0,0,x) - Jacobi(n-1,0,0,x)
        return test





    def train(self,nIter,total_record):
     #maybe like variables sess.run will do the rest, u(boundary cond) is assigned to u_tf ,x(boundary) is assigned to x_tf and so on
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
        lossb_weight=1.0
        start_time= time.time()

        for epoch in range(nIter+1):
                
                with tf.GradientTape() as tape:
                    tape.watch(self.trainable_variables)
                    loss_valueb,loss_valuev=self.calculate_loss()
                    loss=tf.add(loss_valueb,loss_valuev)

                
                gradients = tape.gradient(loss,self.trainable_variables)

                optimizer.apply_gradients(zip(gradients,self.trainable_variables))


                # Print training loss for this epoch
                if (epoch) % 100 == 0:
                            elapsed = time.time() - start_time
                            str_print = 'It: %d, Lossb: %.3e, Lossv: %.3e, Time: %.2f'
                            print(str_print % (epoch, loss_valueb, loss_valuev, elapsed))
                            start_time = time.time()        
                            total_record.append(np.array([epoch, loss,loss_valueb,loss_valuev]))
        return total_record
    

    def print(self):
         print(self.trainable_variables)
    

    @tf.function
    def calculate_loss(self,):
        varloss_total = 0.0
        for e in range(N_Element):
            F_ext_element  = self.F_ext_total[e]
            Ntest_element  = np.shape(F_ext_element)[0] #for each element of the grid you have a vector of the focing term (suppose its n-loc),so the you can have at most quad formula n_loc
            
            x_quad_element = tf.convert_to_tensor(self.grid[e] + (self.grid[e+1]-self.grid[e])/2*(self.x_quad+1),tf.float64)
            #to change change integral to the ref segment in (-1,1)
            jacobian= (self.grid[e+1]-self.grid[e])/2

            x_quad_element = tf.convert_to_tensor(self.grid[e] + (self.grid[e+1]-self.grid[e])/2*(self.x_quad+1),tf.float64)
            x_b_element    = tf.convert_to_tensor(np.array([[self.grid[e]], [self.grid[e+1]]]),tf.float64)


            #test_quad_element =tf.convert_to_tensor(self.Test_fcn(Ntest_element, x_quad),tf.float64)
            #d1test_quad_element, d2test_quad_element = tf.convert_to_tensor(self.dTest_fcn(Ntest_element, x_quad),tf.float64)

            #d1test_bound_element, d2test_bounda_element = tf.convert_to_tensor(self.dTest_fcn(Ntest_element, np.array([[-1],[1]])),tf.float64)



            d1u_NN_quad_element=self.net_du(x_quad_element)

            """
            with tf.GradientTape() as tape_:
                tape_.watch(x_quad_element)
                y=self(x_quad_element)
            d1u_NN_quad_element=tape_.gradient(y,x_quad_element)
            """






            U_NN_element = tf.reshape(tf.stack([tf.reduce_sum(self.w_quad*d1u_NN_quad_element*d1test_quad_element[i]) \
                                    for i in range(Ntest_element)]),(-1,1)) 
         



            Res_NN_element = U_NN_element - F_ext_element

        
            loss_element = tf.reduce_mean(tf.square(Res_NN_element))

            varloss_total = varloss_total+loss_element

        lossb = tf.reduce_mean(tf.square(self(X_bound) -u_bound))   #u_NN_pred is what your network has calc,while u_tf is the real value
        return lossb,varloss_total



In [11]:

model=VPINN(F_ext_total,x_quad,w_quad,grid,test_quad_element,d1test_quad_element, d2test_quad_element,d1test_bound_element, d2test_bound_element)
tf.print(model.trainable_variables)

()


In [12]:
toral_record=model.train(5000,[])

It: 0, Lossb: 7.891e-01, Lossv: 3.230e+03, Time: 4.56
It: 100, Lossb: 9.976e-01, Lossv: 8.659e+02, Time: 3.65
It: 200, Lossb: 9.982e-01, Lossv: 8.659e+02, Time: 3.78
It: 300, Lossb: 9.982e-01, Lossv: 8.659e+02, Time: 3.80
It: 400, Lossb: 9.982e-01, Lossv: 8.659e+02, Time: 3.84
It: 500, Lossb: 9.982e-01, Lossv: 8.659e+02, Time: 4.03
It: 600, Lossb: 9.982e-01, Lossv: 8.659e+02, Time: 3.96
It: 700, Lossb: 9.982e-01, Lossv: 8.659e+02, Time: 4.00
It: 800, Lossb: 9.982e-01, Lossv: 8.659e+02, Time: 4.72
It: 900, Lossb: 9.982e-01, Lossv: 8.659e+02, Time: 4.40
It: 1000, Lossb: 9.982e-01, Lossv: 8.659e+02, Time: 4.19
It: 1100, Lossb: 9.982e-01, Lossv: 8.659e+02, Time: 4.02
It: 1200, Lossb: 9.982e-01, Lossv: 8.659e+02, Time: 4.15
It: 1300, Lossb: 9.982e-01, Lossv: 8.659e+02, Time: 4.42
It: 1400, Lossb: 9.982e-01, Lossv: 8.659e+02, Time: 4.57
It: 1500, Lossb: 9.982e-01, Lossv: 8.659e+02, Time: 4.30
It: 1600, Lossb: 9.982e-01, Lossv: 8.659e+02, Time: 4.69
It: 1700, Lossb: 9.982e-01, Lossv: 8.659e+0

In [ ]:
print(model.trainable_variables)

(<tf.Variable 'dense/kernel:0' shape=(1, 20) dtype=float64, numpy=
array([[-0.39829211, -0.10413188,  0.07586817, -0.35831322, -0.33167458,
        -0.04994269,  0.03266455, -0.255864  ,  0.07028182, -0.21159276,
         0.05855876,  0.01711553, -0.18762854,  0.32170792, -0.29634053,
        -0.48469646, -0.23175118, -0.11371145, -0.07847884, -0.21133142]])>, <tf.Variable 'dense/bias:0' shape=(20,) dtype=float64, numpy=
array([-0.01371339, -0.01371632, -0.01368483, -0.01376364, -0.00094108,
       -0.00092767, -0.01368729,  0.01367991, -0.01162568,  0.01371064,
        0.013713  ,  0.00049088, -0.01373196,  0.01374125, -0.01377428,
       -0.00082772,  0.00015434, -0.01376725,  0.00053878, -0.01374388])>, <tf.Variable 'dense_1/kernel:0' shape=(20, 20) dtype=float64, numpy=
array([[-0.2867928 , -0.07977509,  0.06133655, -0.2608583 , -0.24113989,
        -0.05290179,  0.02272835, -0.18789985,  0.04837921, -0.15104669,
         0.04554949,  0.03902114, -0.13852979,  0.24924868, -0.215912